# Data Exporting
(fixing)

**CAVEAT**: This part is being fixed. It will be available soon, so stay tuned!

In [5]:
import os
import json
import numpy as np
import pandas as pd
from pathlib import Path

In [1]:
from cytokinin.data import take_data

INFO:root:CYTOKININ LOADED


In [6]:
root = Path('./../') # quite the same as str
IMGS = root.joinpath('./cytokinin/cytokinin/tests/mocks/imgs/')

## Export

In [9]:
# Create a Data from two roots: dogs and stones
dogs = take_data('images').store_filesnames_from_folder(IMGS.joinpath('dog'))
stones = take_data('images').store_filesnames_from_folder(IMGS.joinpath('stone'))
dands = dogs.copy().add_from_data(stones)
print(dands)

Object <class 'cytokinin.data.Data'>, of data type "images"
,             75 file paths stored
             0 labels stored.


#### to Keras ImageDataGenerator

In [12]:
dogs.export_to_keras()

Exception: In order to use this function you need to install     "tensorflow" module. Visit https://pypi.org/project/tensorflow/

it also accepts arguments

In [13]:
idg = {
    'rescale': 1./255,
    'shear_range': 0.2,
    'zoom_range': 0.2,
    'horizontal_flip': True
}
ke_gen = dogs.export_to_keras(imagedatagenerator_args=idg)

Exception: In order to use this function you need to install     "tensorflow" module. Visit https://pypi.org/project/tensorflow/

In [70]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import SGD, Adam

self = dands
xcol_name='filename'
ycol_name='class'
ycol=None
withAbsPath=False
y_as = '1hot'

xcolName = self.name
if xcol_name and type(xcol_name) == str: xcolName = xcol_name
df = self.filesnames.to_frame()
if not ycol:
    df['y'] = self.name # no ycol -> assume all x are from one category
# if we have a ycol -> append it as y col
if type(ycol) == list or type(ycol) == np.ndarray:
    if len(ycol) != len(self.filesnames):
        raise Exception('WrongArgument: ycol length is different from self.filesnames length')
    df['y'] = np.array(ycol)
if y_as == 'str':
    df['y'] = df['y'].astype(str)
if y_as == 'categorical':
    df['y'] = df['y'].astype("category")
if y_as == 'classnum':
    df['y'] == df['y'].astype("category").cat.codes
# if y_as == '1hot':
#     df['y'] == pd.get_dummies(df['y']).values
# if withAbsPath:
#     to_abs_path = lambda x: os.path.abspath(x) if type(x) != pathlib.PosixPath else x.absolute()
#     df[xcolName] = df[xcolName].map(to_abs_path)

# df.rename(columns={self.name: xcolName, 'y': ycol_name}, inplace=True)  
# df.head()


# from tensorflow.keras.preprocessing.image import ImageDataGenerator 
# dg = ImageDataGenerator()
# ke_gen = dg.flow_from_dataframe(
#             dataframe = df,
#             class_mode='categorical'
#         )

# model.fit_generator(
#         ke_gen,
#         #steps_per_epoch=20,
#         epochs=3,
#         #validation_data=validation_generator,
#         validation_steps=1)

In [79]:
dands.name, dogs.name, stones.name

('data15776388328336146', 'data15776388328336146', 'data15776388328355122')

In [75]:
df['z'] = pd.get_dummies(df['y']).values
df.y.unique()
# df.iloc[0]

array(['data15776388328336146'], dtype=object)

In [50]:
df.iloc[0]['class']

'0     data15776388328336146\n1     data15776388328336146\n2     data15776388328336146\n3     data15776388328336146\n4     data15776388328336146\n5     data15776388328336146\n6     data15776388328336146\n7     data15776388328336146\n8     data15776388328336146\n9     data15776388328336146\n10    data15776388328336146\n11    data15776388328336146\n12    data15776388328336146\n13    data15776388328336146\n14    data15776388328336146\n15    data15776388328336146\n16    data15776388328336146\n17    data15776388328336146\n18    data15776388328336146\n19    data15776388328336146\n20    data15776388328336146\n21    data15776388328336146\n22    data15776388328336146\n23    data15776388328336146\n24    data15776388328336146\n25    data15776388328336146\n26    data15776388328336146\n27    data15776388328336146\n28    data15776388328336146\n29    data15776388328336146\n30    data15776388328336146\n31    data15776388328336146\n32    data15776388328336146\n33    data15776388328336146\n34    data157

In [30]:
dands.filesnames

0     ./cytokinin/cytokinin/tests/mocks/imgs/dog/dog...
1     ./cytokinin/cytokinin/tests/mocks/imgs/dog/dog...
2     ./cytokinin/cytokinin/tests/mocks/imgs/dog/dog...
3     ./cytokinin/cytokinin/tests/mocks/imgs/dog/dog...
4     ./cytokinin/cytokinin/tests/mocks/imgs/dog/dog...
                            ...                        
35    ./cytokinin/cytokinin/tests/mocks/imgs/stone/s...
36    ./cytokinin/cytokinin/tests/mocks/imgs/stone/s...
37    ./cytokinin/cytokinin/tests/mocks/imgs/stone/s...
38    ./cytokinin/cytokinin/tests/mocks/imgs/stone/s...
39    ./cytokinin/cytokinin/tests/mocks/imgs/stone/s...
Name: data15776379149191175, Length: 75, dtype: object

In [34]:
### Fit a Keras model ####

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import SGD, Adam

n_classes = 1

### Model definition ###
model = Sequential()
model.add( Dense(100, activation='relu', input_shape=(256,256,3)) )
model.add( Dense(60, activation='relu') )
model.add( Dense(10, activation='softmax') )
model.add( Flatten() )
model.add( Dense(n_classes) )

### Compile ###
sgd = SGD(lr=1e-4, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
imsize = (64,64)

### Fit ###
model.fit_generator(
        ke_gen,
        #steps_per_epoch=20,
        epochs=3,
        #validation_data=validation_generator,
        validation_steps=1)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
DEBUG:PIL.PngImagePlugin:STREAM b'IHDR' 16 13
DEBUG:PIL.PngImagePlugin:STREAM b'iCCP' 41 292
DEBUG:PIL.PngImagePlugin:iCCP profile name b'ICC Profile'
DEBUG:PIL.PngImagePlugin:Compression method 0
DEBUG:PIL.PngImagePlugin:STREAM b'IDAT' 345 65536
DEBUG:PIL.Image:Error closing: 'Image' object has no attribute 'fp'
DEBUG:PIL.Image:Error closing: 'Image' object has no attribute 'fp'
DEBUG:PIL.Image:Error closing: 'Image' object has no attribute 'fp'
DEBUG:PIL.Image:Error closing: 'Image' object has no attribute 'fp'
DEBUG:PIL.Image:Error closing: 'Image' object has no attribute 'fp'
DEBUG:PIL.Image:Error closing: 'Image' object has no attribute 'fp'
DEBUG:PIL.PngImagePlugin:STREAM b'IHDR' 16 13
DEBUG:PIL.PngImagePlugin:STREAM b'iCCP' 41 3142
DEBUG:PIL.PngImagePlugin:iCCP profile name b'ICC Prof

Epoch 1/3


ValueError: You are passing a target array of shape (3, 1) while using as loss `categorical_crossentropy`. `categorical_crossentropy` expects targets to be binary matrices (1s and 0s) of shape (samples, classes). If your targets are integer classes, you can convert them to the expected format via:
```
from keras.utils import to_categorical
y_binary = to_categorical(y_int)
```

Alternatively, you can use the loss function `sparse_categorical_crossentropy` instead, which does expect integer targets.

#### to FastAI

In [ ]:
c.length()

In [ ]:
c.filesnames_to_ml_df().head()

In [ ]:
from fastai.vision import ImageDataBunch,ImageList
from fastai.vision.transform import get_transforms
# https://docs.fast.ai/vision.data.html#ImageDataBunch.from_df
# fastai_gen = ImageDataBunch.from_df(path, df, ds_tfms=tfms, size=24)
tfms_arg = get_transforms() # https://docs.fast.ai/vision.transform.html
idb_args = {
    'ds_tfms': tfms_arg,
    'valid_pct': 0,
    'bs': 40,
#     'label_col': None, #label column
}
#fastai_idb = ImageDataBunch.from_df(path='',df=c.filesnames_to_ml_df(), valid_pct=0, bs=40)


fastai_idb = a.export_to_fastAI(imagedatabunch_args=idb_args)
fastai_idb

In [ ]:
type(fastai_idb)

In [ ]:
fastai_idb.open( fastai_idb.items[1] )

#### to Pytorch

In [ ]:
#https://stanford.edu/~shervine/blog/pytorch-how-to-generate-data-parallel

In [ ]:
doggo = './cytokinin/tests/mocks/imgs/dog'

splitta = lambda x: os.path.split(x)[0]
paths = list(set(c.filesnames.apply(splitta)))
paths
# iml1 = ImageList.from_df(doggo);
# iml1

In [ ]:
c.filesnames.apply(splitta)

In [ ]:
a = pd.DataFrame()
a['c1'] = c.filesnames
a['c2'] = c.filesnames
a.head()

In [ ]:
c.filesnames

DEBUG:PIL.Image:Error closing: 'Image' object has no attribute 'fp'
DEBUG:PIL.Image:Error closing: 'Image' object has no attribute 'fp'
DEBUG:PIL.Image:Error closing: 'Image' object has no attribute 'fp'
DEBUG:PIL.Image:Error closing: 'Image' object has no attribute 'fp'
DEBUG:PIL.PngImagePlugin:STREAM b'IHDR' 16 13
DEBUG:PIL.PngImagePlugin:STREAM b'gAMA' 41 4
DEBUG:PIL.PngImagePlugin:STREAM b'cHRM' 57 32
DEBUG:PIL.PngImagePlugin:STREAM b'bKGD' 101 6
DEBUG:PIL.PngImagePlugin:b'bKGD' 101 6 (unknown)
DEBUG:PIL.PngImagePlugin:STREAM b'pHYs' 119 9
DEBUG:PIL.PngImagePlugin:STREAM b'tIME' 140 7
DEBUG:PIL.PngImagePlugin:b'tIME' 140 7 (unknown)
DEBUG:PIL.PngImagePlugin:STREAM b'IDAT' 159 32768
DEBUG:PIL.PngImagePlugin:STREAM b'tEXt' 152600 37
DEBUG:PIL.PngImagePlugin:STREAM b'tEXt' 152649 37
DEBUG:PIL.PngImagePlugin:STREAM b'tEXt' 152698 36
DEBUG:PIL.Image:Error closing: 'Image' object has no attribute 'fp'
DEBUG:PIL.Image:Error closing: 'Image' object has no attribute 'fp'
DEBUG:PIL.Image:Err